In [1]:
import pandas as pd

In [3]:
fpath = 'data/prep/'
dfile = 'form_data.csv'

In [5]:
df = pd.read_csv( fpath + dfile, header=0, parse_dates=True )

In [7]:
df.head()

,RaceId,CourseName,RaceDate,RaceTime,Class,Prize,FieldSize,Distance,RaceWinTime,Posn,...,RT,AgeMin,AgeMax,AW,Going1,Going1N,Won,Placed,ReturnWin,ReturnPlace
0,42370101,Catterick,2016-01-01,12:30:00,4.0,5198.0,12,4246,305.0,1,...,NvH,4.0,NaN,False,soft,5.0,True,0.2,5.5,1.90
1,42370101,Catterick,2016-01-01,12:30:00,4.0,5198.0,12,4246,305.0,2,...,NvH,4.0,NaN,False,soft,5.0,False,0.2,0.0,1.80
2,42370101,Catterick,2016-01-01,12:30:00,4.0,5198.0,12,4246,305.0,3,...,NvH,4.0,NaN,False,soft,5.0,False,0.2,0.0,1.22
3,42370101,Catterick,2016-01-01,12:30:00,4.0,5198.0,12,4246,305.0,4,...,NvH,4.0,NaN,False,soft,5.0,False,0.0,0.0,0.00
4,42370101,Catterick,2016-01-01,12:30:00,4.0,5198.0,12,4246,305.0,5,...,NvH,4.0,NaN,False,soft,5.0,False,0.0,0.0,0.00


In [17]:
pd.DataFrame([df.columns, df.dtypes, df.isna().sum()]).T

,0,1,2
0,RaceId,int64,0
1,CourseName,object,0
2,RaceDate,object,0
3,RaceTime,object,0
4,Class,float64,270942
5,Prize,float64,0
6,FieldSize,int64,0
7,Distance,int64,0
8,RaceWinTime,float64,164
9,Posn,object,0


What features (by row, to date):
* Can be engineered from these data (for 2016-24), and
* Can be obtained quickly on the day from RP, and
* Have a bearing on the outcome?

In [23]:
# trainer/jockey
# GB Flat last 5 seasons (ie current_year-4 to current_year)
# GB Flat current season
# IRE Flat last 5 seasons (ie current_year-4 to current_year)
# IRE Flat current season
## 2yo AW nruns, nwins, nplaced, earnings
## ...
# last 14 days: nruns/rides, nwins

Runner features:
* Age
* 

### RP Stats box


Can I calculate, for every runner, in every race (filtered on flat), the number of previous wins/runs, for the present going, distance, course?

I.e., I need to be able to calculate those numbers (in the RP Stats box) historically, and then see which ones are relevant (by course, distance, going, class/prize).

In [32]:
# filter on flat
df.RT.unique()

array(['NvH', 'HcH', 'NvCh', 'HcCh', 'H', 'NHF', 'NvHcH', 'MdH', 'Ch',
       'MdNHF', 'MdHM', 'NvHcCh', 'Hc', 'Md', '', 'HcF', 'HM', 'MdF',
       'ChM', 'NHFM', 'NvHM', 'NvHcChM', 'HcHM', 'HF', 'NvChM', 'F',
       'NHFF', 'MdCh', 'MdNHFM', 'M', 'MdChM', 'Nv', 'NvHcHM', 'NvF',
       'HcChM', 'HcHF', 'HcM', 'NvHc', 'NvHF', 'MdHF', 'ChF', 'HcNHF',
       'NvNHF', 'NvNHFF', 'MdM', 'NvNHFM', 'NvM', 'NvHcHF', 'MdNHFF',
       'NvChF'], dtype=object)

In [30]:
# nan's in RT should be ''
df.loc[df.RT.isna(), 'RT'] = ''

In [46]:
# flat races are: {'', 'Hc', 'Md', 'Nv'} %+% {'', 'F', 'M'} or 'NvHc'
flat = [ x + y for x in ['','Hc','Md','Nv'] for y in ['','F','M'] ] + ['NvHc']

In [50]:
flat_df = df[df.RT.apply(lambda rt: rt in flat)]